## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
attrition_df.isnull().sum()

,0
Age,0
Attrition,0
BusinessTravel,0
Department,0
DistanceFromHome,0
Education,0
EducationField,0
EnvironmentSatisfaction,0
HourlyRate,0
JobInvolvement,0


In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [5]:
y_df.shape

(1470, 2)

In [6]:
sel_columns = ['Education',
               'Age',
               'DistanceFromHome',
               'JobSatisfaction',
               'OverTime',
               'StockOptionLevel',
               'WorkLifeBalance',
               'YearsAtCompany',
               'YearsSinceLastPromotion',
               'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[sel_columns]

# Show the data types for X_df
X_df.dtypes

,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [7]:
X_df.shape

(1470, 10)

In [8]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)
y_train.shape


(1102, 2)

In [9]:
from sklearn.preprocessing import OneHotEncoder
overtime_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
overtime_encoder.fit(np.array(X_train['OverTime']).reshape(-1, 1))
X_train['OverTimeEncoded'] = overtime_encoder.transform(np.array(X_train['OverTime']).reshape(-1, 1))
X_test['OverTimeEncoded'] = overtime_encoder.transform(np.array(X_test['OverTime']).reshape(-1, 1))

X_train = X_train.drop(columns='OverTime')
X_test = X_test.drop(columns='OverTime')

print(X_train.dtypes)
print(X_test.dtypes)

Education                    int64
Age                          int64
DistanceFromHome             int64
JobSatisfaction              int64
StockOptionLevel             int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
NumCompaniesWorked           int64
OverTimeEncoded            float64
dtype: object
Education                    int64
Age                          int64
DistanceFromHome             int64
JobSatisfaction              int64
StockOptionLevel             int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
NumCompaniesWorked           int64
OverTimeEncoded            float64
dtype: object


In [10]:
X_train.shape

(1102, 10)

In [11]:
# Create a StandardScaler
# Fit the StandardScaler to the training data
scaler = StandardScaler().fit(X_train)


# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


y_train.shape


(1102, 2)

In [12]:
y_train

,Attrition,Department
464,No,Research & Development
512,No,Research & Development
480,Yes,Sales
680,No,Research & Development
1023,No,Research & Development
...,...,...
715,No,Research & Development
905,No,Research & Development
1096,No,Human Resources
235,No,Sales


In [13]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
department_encoder.fit(np.array(y_train['Department']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_department_train = department_encoder.transform(np.array(y_train['Department']).reshape(-1, 1))
y_department_test = department_encoder.transform(np.array(y_test['Department']).reshape(-1, 1))

y_department_train

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [14]:
y_train.shape

(1102, 2)

In [15]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
attrition_encoder.fit(np.array(y_train['Attrition']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train = attrition_encoder.transform(np.array(y_train['Attrition']).reshape(-1, 1))
y_attrition_test = attrition_encoder.transform(np.array(y_test['Attrition']).reshape(-1, 1))

y_attrition_train

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [16]:
# Find the number of columns in the X training data
number_input_features = len(X_train_scaled[0])
number_input_features

# Create the input layer
input_layer = layers.Input(shape=(X_train_scaled.shape[1],), name='input_features')

# Create at least two shared layers
shared_layer_1 = layers.Dense(units=64, activation='relu')(input_layer)
shared_layer_2 = layers.Dense(32, activation='relu')(shared_layer_1)

In [17]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(units=16, activation='relu')(shared_layer_2)

# Create the output layer
department_output = layers.Dense(units=y_department_train.shape[1], activation='sigmoid', name="department_output")(department_hidden)


In [18]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(units=16, activation='relu')(shared_layer_2)

# Create the output layer
attrition_output = layers.Dense(units=y_attrition_train.shape[1], activation='sigmoid', name="attrition_output")(attrition_hidden)


In [19]:
# @title Default title text
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={
                  'department_output': 'categorical_crossentropy',
                  'attrition_output': 'binary_crossentropy'
              },
              metrics={
                  'department_output': 'accuracy',
                  'attrition_output': 'accuracy'
              })

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_features            │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │            704 │ input_features[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             51 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             34 │ dense_3[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,925 (15.33 KB)

 Trainable params: 3,925 (15.33 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Train the model
fit_model = model.fit(X_train_scaled, {'department_output': y_department_train, 'attrition_output': y_attrition_train}, epochs=100)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - attrition_output_accuracy: 0.6607 - department_output_accuracy: 0.5122 - loss: 1.6450
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - attrition_output_accuracy: 0.8515 - department_output_accuracy: 0.6540 - loss: 1.2914
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - attrition_output_accuracy: 0.8468 - department_output_accuracy: 0.6425 - loss: 1.2227
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8390 - department_output_accuracy: 0.6533 - loss: 1.1772
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8268 - department_output_accuracy: 0.6677 - loss: 1.1583
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8584 - department_output_accuracy: 0.6674 - loss: 1.0996
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8716 - department_output_accuracy: 0.6583 - loss: 1.0650
Epoch 8/100
35/35 ━━━━━━━━

In [21]:
# Evaluate the model with the testing data
test_results = model.evaluate(
    X_test_scaled,
    {'department_output': y_department_test, 'attrition_output': y_attrition_test},
    verbose=2
)

12/12 - 0s - 23ms/step - attrition_output_accuracy: 0.7935 - department_output_accuracy: 0.4973 - loss: 2.4483


In [26]:
attrition_accuracy, department_accuracy, loss = model.evaluate(
    X_test_scaled,
    {'department_output': y_department_test, 'attrition_output': y_attrition_test},
    verbose=2
)

print(f"Department Accuracy: {department_accuracy}")
print(f"Attrition Accuracy: {attrition_accuracy}")

12/12 - 0s - 6ms/step - attrition_output_accuracy: 0.7935 - department_output_accuracy: 0.4973 - loss: 2.4483
Department Accuracy: 0.79347825050354
Attrition Accuracy: 2.448272466659546


In [25]:
# Print the accuracy for both department and attrition
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[1]}")

# Print the accuracy for the attrition output
print(f"Attrition Accuracy: {test_results[2]}")

Department Accuracy: 0.79347825050354
Attrition Accuracy: 0.49728259444236755


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric to use on this data, especially for the attrition prediction, because the dataset might be imbalanced. This means there might be significantly more employees who stayed than those who left.
2. I used sigmoid and it's the best for the activation layer this time around because it's solving a binary classification problem.
3. Hyperparameter Tuning:
One improvement is fine-tuning hyperparameters, such as the number of layers, units, or the activation function. I would use GridSearchCV to automate this process and find the best settings for the model. This helps improve its performance without relying on trial and error.

Address Class Imbalance:
If there’s a class imbalance, like more employees who stayed than left, I would use oversampling or undersampling to balance the dataset. This reduces bias and improves prediction accuracy for employees likely to leave.

Feature Engineering:
I would create or modify features to give the model better insights. For example, combining "YearsAtCompany" with "YearsSinceLastPromotion" could highlight career progression, which might improve the model's predictions on employee attrition.